In [1]:
import pandas as pd
import numpy as np
import time
from fp.fp import FreeProxy

In [2]:
okru_url = "https://ok.ru/video"

In [3]:
CHROMEDRIVER_PATH = "/Users/timdunn/Downloads/chromedriver_mac_arm64/chromedriver"
WINDOW_SIZE = "1920,1080"

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [5]:
def go_to_startpage(driver, start_url):
    driver.get(okru_url)
    return driver

In [6]:
def get_search_button(driver):
    element = driver.find_element(By.CLASS_NAME, "wrap-input__414z3")
    return element.find_element(By.TAG_NAME, "button")

In [7]:
# <h3 class="portlet_h_name_t">Weekly hits</h3>
def on_default_page(driver):
    try:
        return driver.find_element(By.CLASS_NAME, "portlet_h_name_t").text == "Weekly hits"
    except:
        return False

In [8]:
def popup_available(driver):
    try:
        popup(driver)
        return True
    except:
        return False

In [9]:
def popup(driver):
    return driver.find_element(By.CLASS_NAME, "close-button__akasx")

In [10]:
def close_popup(driver):
    if popup_available(driver):
        popup(driver).click()

In [11]:
def video_search(driver):
    return driver.find_element(By.XPATH, "//input[@placeholder='Video search']")

In [12]:
def search_movie(driver, movie_name, year):
    video_search(driver).send_keys("{} ({})".format(movie_name, year), Keys.RETURN)
#     driver.execute_script("arguments[0].click();", get_search_button(driver))
#     get_search_button(driver).click()
#     video_search(driver).send_keys("{} ({})".format(movie_name, year))
#     video_search(driver).submit()

In [13]:
def any_search_results(driver):
    try :
        return not (driver.find_element(By.CLASS_NAME, "stub-empty_t").text == 'No search results found')
    except:
        return True

In [14]:
def all_titles_movies(driver):
    return [movie.text for movie in driver.find_elements(By.CLASS_NAME, "video-card_duration")]

In [15]:
def movie_dict(element, tconst):
    return {
        "link title": get_movie_link_title(element),
        "link url": get_movie_url(element),
        "movie duration": get_movie_duration(element),
        "tconst": tconst
    }

In [16]:
def get_movie_link_title(element):
    try:
        return element.find_element(By.CLASS_NAME, "video-card_n").get_attribute("text")
    except:
#         print("movie title exception")
        return None

In [17]:
def get_movie_url(element):
    try:
#         print("movie url is", element.get_attribute("data-id"))
        return element.get_attribute("data-id")
    except:
#         print("movie url exception block")
        return None

In [18]:
def get_movie_duration(element):
    try:
#         print("movie duration is", element.find_element(By.CLASS_NAME, "video-card_duration").get_attribute("text"))
        return element.find_element(By.CLASS_NAME, "video-card_duration").text
    except:
#         print("movie duration exception block")
        return None

In [19]:
def get_all_movie_info(driver, tconst):
    try:
        return [movie_dict(element, tconst) for element in driver.find_elements(By.CLASS_NAME, "video-card")]
    except:
        print("all movie info except block")
        return []

In [20]:
def scroll_for_all_results(driver):
    movie_results = 0
    current_results = 1
    while movie_results != current_results:
        movie_results = current_results
        close_popup(driver)
        scroll_bottom(driver)
        time.sleep(3)
        current_results = len(all_titles_movies(driver))
    return current_results

In [21]:
def scroll_bottom(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [22]:
def get_movie_search_results(driver, movie_name, year, tconst, instance_id, 
                             index, okru_url="https://ok.ru/video", all_data=[]):
    go_to_startpage(driver, okru_url)
    close_popup(driver)
    video_search(driver).clear()
    time.sleep(3)
    basic_results = all_titles_movies(driver)
    close_popup(driver)
    search_movie(driver, movie_name, year)
    close_popup(driver)
    if any_search_results(driver):
        close_popup(driver)
        time.sleep(5)
        if on_default_page(driver):
            print("on default page if")
            return "pending"
#             return restart_movie_search(driver, movie_name, year, tconst, okru_url, all_data)
        else:
            if index % 10 == 0:
                print("Went into else of instance {} and index {}".format(instance_id, index))
            scroll_for_all_results(driver)
            close_popup(driver)
            results = get_all_movie_info(driver, tconst)
            if len(results) == 0:
                return [{'link title': None,'link url': None,'movie duration': None,'tconst': tconst}]
            else:
                return results
    else:
        no_match_dict = {'link title': None,'link url': None,'movie duration': None,'tconst': tconst}
        return [no_match_dict]

In [23]:
def restart_movie_search(driver, movie_name, year, tconst, okru_url, all_data):
    try:
        driver.quit()
        driver = get_driver()
        go_to_startpage(driver, okru_url)
        close_popup(driver)
        return get_movie_search_results(driver, movie_name, year, tconst, okru_url, all_data)
    except:
        no_match_dict = {'link title': None,'link url': None,'movie duration': None,'tconst': tconst}
        return [no_match_dict]

In [24]:
from all_processes.update_movie_data import *

In [25]:
imdb_df = UpdateMovieData().get_latest_imdb()

/Users/timdunn/Desktop/imdb_latest/name.basics.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.akas.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.basics.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.crew.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.episode.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.principals.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.ratings.tsv.gz downloaded!



In [24]:
imdb_df = pd.read_csv("/Users/timdunn/Downloads/title.basics (1)-1.tsv", sep="\t")

/var/folders/z9/ml790yl90yj8nyc5j320wb4w0000gn/T/ipykernel_1520/1203546268.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv("/Users/timdunn/Downloads/title.basics (1)-1.tsv", sep="\t")


In [43]:
imdb_df = imdb_df.loc[imdb_df["titleType"].isin(["movie", "tvMovie", "video"])]

In [45]:
dfs = [pd.read_csv(file_path) for file_path in listdir_nohidden("/Users/timdunn/Desktop/okru_checkpoints/already used/")]

In [46]:
okru_progress_path = "/Users/timdunn/Desktop/okru_progress.csv"
dfs.append(pd.read_csv(okru_progress_path))
final_df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)

In [47]:
final_df.to_csv(okru_progress_path, index=False)

In [49]:
final_df.drop_duplicates(subset=['link url'])

,link title,link url,movie duration,tconst,duration,year,searchable_movie_name
0,Smile Before Death (1972),3.444223e+12,1:28:15,tt0069301,NaN,NaN,NaN
1,Эта весёлая планета (1973),NaN,NaN,tt0070042,NaN,NaN,NaN
5,Привести в исполнение (1973) / Executive Actio...,1.824677e+12,1:27:05,tt0070046,NaN,NaN,NaN
6,"Executive Action (1973) \tBurt Lancaster, Robe...",2.278275e+12,1:27:05,tt0070046,NaN,NaN,NaN
7,Burt Lancaster Executive Action (1973) DVDRip ...,1.642334e+12,1:26:48,tt0070046,NaN,NaN,NaN
...,...,...,...,...,...,...,...
19611961,Альфред Бестер Адам без Евы (Фантастический р...,1.242558e+12,NaN,NaN,0.0,1941.0,альфред бестер адам без евы (фантастический ра...
19611962,Русский художник Анненков Юрий Павлович,5.558962e+09,NaN,NaN,0.0,NaN,русский художник анненков юрий павлович
19611963,"NWO: Die EU Maske fällt. Erstes Land führt ""TO...",1.099150e+10,NaN,NaN,0.0,NaN,nwo die eu maske fällt erstes land führt tole...
19611964,"Любовь удивляет / Hasard et l'amour, Le.1913.",1.550677e+12,NaN,NaN,0.0,1913.0,"любовь удивляет / hasard et lamour, le 1913"


In [74]:
from requirements import *

In [ ]:
imdb_df.index

In [26]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=%s" % WINDOW_SIZE)
#     proxy = FreeProxy(elite=True).get()
#     options.add_argument('--proxy-server=%s' % proxy)
    return webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=options)

In [27]:
def get_tconst_okru():
    return list(pd.read_csv("/Users/timdunn/Desktop/newokru.csv")["tconst"])

In [28]:
def filter_imdb(imdb_df, okru_tconst):
    return imdb_df.loc[~imdb_df["tconst"].isin(okru_tcosnt)]

In [ ]:
imdb_df.loc[imdb_df["tconst"] == "tt0004256"]

In [ ]:
other_data = []

In [29]:
def convert_data(the_data):
    the_data = flatten(the_data)
    the_data = list(filter(has_some_data, the_data))
#     print(the_data)
    return list(filter(lambda data: type(data) == dict, the_data))

In [30]:
from selenium.webdriver.chrome.options import Options

In [ ]:
driver = get_driver()

In [ ]:
driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=options)

In [31]:
def divide_chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [55]:
def concating_okru_data():
    okru_progress_path = "/Users/timdunn/Desktop/okru_progress.csv"
    dfs = [pd.read_csv(file_path) for file_path in filtered_file_paths("/Users/timdunn/Desktop/okru_checkpoints/")]
    dfs.append(pd.read_csv(okru_progress_path))
    final_df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
    final_df.to_csv(okru_progress_path)
    return final_df

In [56]:
def get_imdb_dfs(imdb_df, instances=5):
    okru_df = concating_okru_data()
    relevant_df = imdb_df.loc[~(imdb_df["runtimeMinutes"] == "\\N") & 
                              ~(imdb_df["tconst"].isin(okru_df["tconst"]))]
    return [relevant_df.loc[indexes] for indexes in divide_chunks(relevant_df.index, 
                                                                  round(relevant_df.index.size/instances))]    

In [48]:
def save_okru_progress_to_df(instance_id, other_data, index):
    if len(other_data) > 0:
        pd.DataFrame(data=other_data).to_csv("/Users/timdunn/Desktop/okru_checkpoints/{}{}.csv".format(
            instance_id, index), index=False)

In [ ]:
imdb_dfs = get_imdb_dfs(imdb_df)

In [49]:
def run_okru_scraper(relevant_df, instance_id, okru_url=okru_url):
    driver = get_driver()
    other_data = []
    try:
        for index, row in relevant_df.iterrows():
            next_data = get_movie_search_results(driver, row["primaryTitle"], row["startYear"], 
                                                 row["tconst"], instance_id, index, okru_url, other_data)
            other_data.append(next_data) 
            if (index % 100 == 0):
#             if (index % 10 == 0):
#                 print(next_data)
#                 print("The length of original data for {} and index {} is {}".format(instance_id, index, len(other_data)))
                print("The length of data for {} and index {} is {}".format(instance_id, index, len(convert_data(other_data))))
                save_okru_progress_to_df(instance_id, convert_data(other_data), index)
        save_okru_progress_to_df(instance_id, convert_data(other_data), index)
    except:
        save_okru_progress_to_df(instance_id, convert_data(other_data), "Exception")
        driver.quit()
        return convert_data(other_data)

In [54]:
def run_instances_okru_scraper(instances, imdb_df):
    imdb_dfs = get_imdb_dfs(imdb_df, instances)
    thread_list = list()
    for instance_id in range(instances):
        print('inside for')
        thread = threading.Thread(name='Test {}'.format(instance_id), target=run_okru_scraper, 
                                  args=(imdb_dfs[instance_id], instance_id))
        thread_list.append(thread)
        thread.start()
        print(thread.name + ' started!')
    for thread in thread_list:
        thread.join()

In [36]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [37]:
def combine_imdb_dfs():
    okru_df = pd.read_csv("/Users/timdunn/Desktop/okru_progress.csv")
    dfs = [pd.read_csv(file_path) for file_path in listdir_nohidden("/Users/timdunn/Desktop/okru_checkpoints/")]
    return pd.concat(flatten([dfs, okru_df])).drop_duplicates().reset_index(
        drop=True).to_csv("/Users/timdunn/Desktop/okru_progress.csv", index=False)

In [ ]:
okru_data = concating_okru_data()

In [ ]:
okru_data.to_csv("/Users/timdunn/Desktop/okru_progress.csv", index=False)

In [ ]:
okru_data = okru_data.drop_duplicates(subset=["link url", "tconst"])

In [ ]:
okru_data = pd.concat([okru_data, pd.read_csv("/Users/timdunn/Desktop/okru_progress.csv")]).drop_duplicates(subset=["link url", "tconst"])

In [88]:
import threading

In [89]:
import glob

In [90]:
import os

In [38]:
def file_has_data(file_path):
    return os.stat(file_path).st_size > 1

In [39]:
def filtered_file_paths(directory):
    return list(filter(file_has_data, listdir_nohidden(directory)))

In [ ]:
paths = filtered_file_paths("/Users/timdunn/Desktop/okru_checkpoints/")

In [ ]:
imdb_df.loc[9500:]

In [95]:

run_instances_okru_scraper(5, imdb_df)



KeyboardInterrupt



In [46]:
try:
    run_instances_okru_scraper(5, imdb_df.loc[9500:])
except:
    print("there was a major exception")

Exception in thread Test 4:
Traceback (most recent call last):
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/Users/timdunn/minif

Exception in thread Test 0:
Traceback (most recent call last):
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/Users/timdunn/miniforge3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/Users/timdunn/minif

there was a major exception


In [1]:
from all_processes.update_movie_data import *

In [2]:
imdb_df = UpdateMovieData().get_latest_imdb()

/Users/timdunn/Desktop/imdb_latest/name.basics.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.akas.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.basics.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.crew.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.episode.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.principals.tsv.gz downloaded!

/Users/timdunn/Desktop/imdb_latest/title.ratings.tsv.gz downloaded!



In [3]:
imdb_df = imdb_df.drop_duplicates(subset=['tconst'])

In [4]:
imdb_df = reset_and_copy(imdb_df.loc[imdb_df['titleType'].isin(['movie', 'tvMovie', 'video'])])

In [5]:
imdb_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,searchable_alias_title,searchable_primary_title,searchable_original_title
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,tt0000009,1,Miss Jerry,\N,\N,original,\N,1,miss jerry,miss jerry,miss jerry
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport",tt0000147,1,The Corbett-Fitzsimmons Fight,US,\N,imdbDisplay,\N,0,the corbett fitzsimmons fight,the corbett fitzsimmons fight,the corbett fitzsimmons fight
2,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,tt0000502,1,Bohemios,\N,\N,original,\N,1,bohemios,bohemios,bohemios
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",tt0000574,10,The Story of the Kelly Gang,AU,\N,imdbDisplay,\N,0,the story of the kelly gang,the story of the kelly gang,the story of the kelly gang
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,tt0000591,1,L'enfant prodigue,FR,\N,imdbDisplay,\N,0,lenfant prodigue,the prodigal son,lenfant prodigue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029549,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,\N,100,Documentary,tt9916680,1,De la ilusión al desconcierto: cine colombiano...,CO,\N,imdbDisplay,\N,0,de la ilusión al desconcierto cine colombiano ...,de la ilusión al desconcierto cine colombiano ...,de la ilusión al desconcierto cine colombiano ...
1029550,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,\N,66,Drama,tt9916692,1,Teatroteka: Człowiek bez twarzy,PL,\N,imdbDisplay,\N,0,teatroteka człowiek bez twarzy,teatroteka czlowiek bez twarzy,teatroteka czlowiek bez twarzy
1029551,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,\N,\N,Comedy,tt9916706,1,Dankyavar Danka,IN,\N,\N,\N,0,dankyavar danka,dankyavar danka,dankyavar danka
1029552,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N,tt9916730,1,६ गुण,IN,hi,imdbDisplay,\N,0,६ गुण,6 gunn,6 gunn


In [ ]:
for num in range(10):
    try:
        run_instances_okru_scraper(9, imdb_df)
    except:
        print("there was a major exception")

inside for
Test 0 started!
inside for
Test 1 started!
inside for
Test 2 started!
inside for
Test 3 started!
inside for
Test 4 started!
inside for
Test 5 started!
inside for
Test 6 started!
inside for
Test 7 started!
inside for
Test 8 started!
Went into else of instance 0 and index 445500
Went into else of instance 4 and index 1555350
The length of data for 0 and index 445500 is 131
Went into else of instance 1 and index 820550
Went into else of instance 2 and index 1146150
Went into else of instance 1 and index 820560
Went into else of instance 3 and index 1367340
Went into else of instance 1 and index 820570
Went into else of instance 3 and index 1367380
Went into else of instance 7 and index 2114710
Went into else of instance 8 and index 2311350
Went into else of instance 1 and index 820590
Went into else of instance 2 and index 1146170
Went into else of instance 5 and index 1737000
Went into else of instance 4 and index 1555390
The length of data for 5 and index 1737000 is 835
Went 

Went into else of instance 1 and index 854280
Went into else of instance 0 and index 447100
The length of data for 0 and index 447100 is 6229
Went into else of instance 6 and index 1928410
Went into else of instance 0 and index 447110
Went into else of instance 3 and index 1368000
The length of data for 3 and index 1368000 is 6055
Went into else of instance 7 and index 2115350
Went into else of instance 1 and index 854320
Went into else of instance 4 and index 1555700
The length of data for 4 and index 1555700 is 7400
Went into else of instance 6 and index 1928430
Went into else of instance 2 and index 1146430
Went into else of instance 2 and index 1146440
Went into else of instance 4 and index 1555710
Went into else of instance 1 and index 854350
Went into else of instance 1 and index 854360
Went into else of instance 2 and index 1146460
Went into else of instance 3 and index 1368060
Went into else of instance 5 and index 1739530
Went into else of instance 5 and index 1739540
Went int

Went into else of instance 3 and index 1368740
Went into else of instance 2 and index 1152210
Went into else of instance 7 and index 2115590
Went into else of instance 4 and index 1556250
Went into else of instance 7 and index 2115610
Went into else of instance 5 and index 1739760
Went into else of instance 2 and index 1152220
Went into else of instance 3 and index 1368770
Went into else of instance 6 and index 1928990
Went into else of instance 1 and index 884290
Went into else of instance 3 and index 1368780
Went into else of instance 7 and index 2115630
Went into else of instance 6 and index 1929000
Went into else of instance 2 and index 1152230
The length of data for 6 and index 1929000 is 19374
Went into else of instance 8 and index 2312940
Went into else of instance 4 and index 1556290
Went into else of instance 3 and index 1368800
Went into else of instance 7 and index 2115650
The length of data for 3 and index 1368800 is 13215
Went into else of instance 7 and index 2115670
Went

Went into else of instance 3 and index 1369220
Went into else of instance 4 and index 1556750
Went into else of instance 2 and index 1152690
Went into else of instance 6 and index 1929360
Went into else of instance 7 and index 2116090
Went into else of instance 2 and index 1152710
Went into else of instance 5 and index 1739930
Went into else of instance 2 and index 1152760
Went into else of instance 8 and index 2313390
Went into else of instance 3 and index 1369230
Went into else of instance 7 and index 2116100
The length of data for 7 and index 2116100 is 21674
Went into else of instance 2 and index 1152770
Went into else of instance 3 and index 1369250
Went into else of instance 6 and index 1929400
The length of data for 6 and index 1929400 is 27826
Went into else of instance 2 and index 1152780
Went into else of instance 5 and index 1739960
Went into else of instance 6 and index 1929410
Went into else of instance 5 and index 1739970
Went into else of instance 3 and index 1369260
Wen

Went into else of instance 2 and index 1153280
Went into else of instance 6 and index 1929510
Went into else of instance 5 and index 1740380
Went into else of instance 1 and index 885560
Went into else of instance 4 and index 1557210
Went into else of instance 5 and index 1740390
Went into else of instance 7 and index 2116620
Went into else of instance 1 and index 885570
Went into else of instance 3 and index 1369490
Went into else of instance 8 and index 2313970
Went into else of instance 3 and index 1369500
The length of data for 3 and index 1369500 is 26916
Went into else of instance 6 and index 1929520
Went into else of instance 1 and index 885590
Went into else of instance 4 and index 1557220
Went into else of instance 7 and index 2116660
Went into else of instance 5 and index 1740400
The length of data for 5 and index 1740400 is 31561
Went into else of instance 2 and index 1153320
Went into else of instance 8 and index 2314000
The length of data for 8 and index 2314000 is 29735
W

...

The length of data for 3 and index 1388800 is 47012
Went into else of instance 3 and index 1388810
Went into else of instance 2 and index 1207180
Went into else of instance 5 and index 1759050
Went into else of instance 1 and index 908830
Went into else of instance 8 and index 2327580
Went into else of instance 1 and index 908860
Went into else of instance 3 and index 1388820
Went into else of instance 1 and index 908870
Went into else of instance 2 and index 1207190
Went into else of instance 7 and index 2131960
Went into else of instance 4 and index 1576050
Went into else of instance 7 and index 2131970
Went into else of instance 1 and index 908900
Went into else of instance 4 and index 1576060
The length of data for 1 and index 908900 is 39857
Went into else of instance 4 and index 1576070
Went into else of instance 5 and index 1759100
The length of data for 5 and index 1759100 is 46024
Went into else of instance 2 and index 1207220
Went into else of instance 7 and index 2131980
Wen

Went into else of instance 8 and index 2327880
Went into else of instance 5 and index 1759400
Went into else of instance 1 and index 909680
The length of data for 5 and index 1759400 is 53748
Went into else of instance 0 and index 495640
Went into else of instance 1 and index 909700
The length of data for 1 and index 909700 is 43667
Went into else of instance 5 and index 1759420
Went into else of instance 2 and index 1207910
Went into else of instance 8 and index 2327900
Went into else of instance 2 and index 1207920
Went into else of instance 1 and index 909710
The length of data for 8 and index 2327900 is 52122
Went into else of instance 6 and index 1947090
Went into else of instance 2 and index 1207930
Went into else of instance 2 and index 1207940
Went into else of instance 3 and index 1389190
Went into else of instance 5 and index 1759430
Went into else of instance 8 and index 2327920
Went into else of instance 6 and index 1947110
Went into else of instance 1 and index 909760
Went

Went into else of instance 5 and index 1759850
Went into else of instance 6 and index 1947550
Went into else of instance 2 and index 1208550
Went into else of instance 6 and index 1947560
Went into else of instance 5 and index 1759860
Went into else of instance 3 and index 1389610
Went into else of instance 1 and index 910060
Went into else of instance 0 and index 496540
Went into else of instance 7 and index 2132980
Went into else of instance 4 and index 1576790
Went into else of instance 2 and index 1208570
Went into else of instance 0 and index 496550
Went into else of instance 1 and index 910070
Went into else of instance 6 and index 1947570
Went into else of instance 2 and index 1208580
Went into else of instance 3 and index 1389620
Went into else of instance 7 and index 2132990
Went into else of instance 4 and index 1576800
Went into else of instance 0 and index 496570
The length of data for 4 and index 1576800 is 55758
Went into else of instance 2 and index 1208590
Went into els

Went into else of instance 0 and index 497950
Went into else of instance 6 and index 1947950
Went into else of instance 8 and index 2328810
Went into else of instance 5 and index 1760160
Went into else of instance 7 and index 2133330
Went into else of instance 2 and index 1209040
Went into else of instance 8 and index 2328820
Went into else of instance 3 and index 1389880
Went into else of instance 2 and index 1209050
Went into else of instance 7 and index 2133360
Went into else of instance 5 and index 1760190
Went into else of instance 1 and index 911080
Went into else of instance 3 and index 1389910
Went into else of instance 6 and index 1947980
Went into else of instance 1 and index 911090
Went into else of instance 2 and index 1209080
Went into else of instance 3 and index 1389920
Went into else of instance 4 and index 1577120
Went into else of instance 5 and index 1760210
Went into else of instance 6 and index 1948010
Went into else of instance 2 and index 1209090
Went into else o

Went into else of instance 5 and index 1760590
Went into else of instance 8 and index 2329260
Went into else of instance 5 and index 1760610
Went into else of instance 4 and index 1577380
Went into else of instance 8 and index 2329280
Went into else of instance 5 and index 1760650
Went into else of instance 1 and index 911680
Went into else of instance 1 and index 911690
Went into else of instance 4 and index 1577390
Went into else of instance 1 and index 911750
Went into else of instance 8 and index 2329360
Went into else of instance 7 and index 2133770
Went into else of instance 4 and index 1577440
Went into else of instance 3 and index 1390660
Went into else of instance 7 and index 2133780


In [45]:
from common_methods import *

In [ ]:
# driver = get_driver()
# tconst = list(filter(has_some_data, all_data))[-1][0]["tconst"]
# first_index = imdb_df.loc[imdb_df["tconst"] == tconst].index[0]
okru_df = pd.read_csv("/Users/timdunn/Desktop/okru_progress.csv")
relevant_df = imdb_df.loc[~(imdb_df["runtimeMinutes"] == "\\N") & ~(imdb_df["tconst"].isin(okru_df["tconst"]))]
for index, row in relevant_df.iterrows():
    next_data = get_movie_search_results(driver, row["primaryTitle"], row["startYear"], row["tconst"], okru_url, other_data)
    other_data.append(next_data) 
    if (index % 100 == 0):
        pd.concat([okru_df, pd.DataFrame(data=convert_data(other_data))]).drop_duplicates().to_csv(
            "/Users/timdunn/Desktop/okru_progress.csv", index=False)

In [ ]:
"/Users/timdunn/Desktop/okru_checkpoints/"

In [ ]:
driver = get_driver()

In [ ]:
go_to_startpage(driver, okru_url)

In [ ]:
close_popup(driver)

In [ ]:
element = driver.find_element(By.CLASS_NAME, "wrap-input__414z3")

In [ ]:
element = driver.find_element(By.CLASS_NAME, "wrap-input__414z3")
button_element = element.find_element(By.TAG_NAME, "button")

In [ ]:
video_search(driver).send_keys("some example")

In [ ]:
button_element.click()

In [ ]:
driver.quit()

In [ ]:
pd.concat([okru_df, pd.DataFrame(data=convert_data(other_data))]).drop_duplicates().to_csv("/Users/timdunn/Desktop/okru_progress.csv", index=False)

In [ ]:
<button type="button" class="button__pe9qo button-core-container__0ej09 squared-button__qz82p"><span class="button-core__0ej09 __size-m__0ej09 __wide__0ej09 squared-button-core__qz82p"><span class="content__0ej09">Search</span><span class="spinner__0ej09"><div class="spinner__paevy __size-20__paevy">
<div class="el__paevy"></div></div></span></span></button>

In [ ]:
other_data[-5:]

In [ ]:
filter(lambda data: data != "pending")

In [ ]:
some_data = all_data[len(all_data) > 0]

In [ ]:
len(all_data)

In [ ]:
all_data[-100:]

In [41]:
def has_some_data(data):
    try:
        return len(data) > 0
    except:
        return False

In [ ]:
all_data[[data != None for data in all_data]]

In [ ]:
filtering = [data != None for data in all_data]
all_data[filtering]

In [ ]:
some_data = list(filter(has_some_data, all_data))

In [ ]:
some_data = flatten(some_data)

In [ ]:
right_data = list(filter(lambda data: type(data) == dict, some_data))

In [ ]:
len(right_data)

In [ ]:
pd.DataFrame(data=right_data).to_csv("/Users/timdunn/Desktop/okru_progress.csv", index=False)

In [ ]:
all_data[len(all_data) > 0]

In [ ]:
imdb_df.loc[~(imdb_df["runtimeMinutes"] == "\\N")]

In [100]:
def update_tconst(movie_hash, row):
    movie_hash["tconst"] = row["tconst"]
    return movie_hash

In [101]:
def update_all_tconst(movie_hashes, row):
    return [update_tconst(movie_hash, row) for movie_hash in movie_hashes]

In [42]:
def flatten(some_list):
    return [item for sublist in some_list for item in sublist]